## Code playground for the lecture block *Analysing High-Dimensional Data*

### 1. PCA

In the following, you can play around with PCA. We will implement a simple version of PCA ourselves and use it to reduce the dimensionality of Pokémon images (from the original GameBoy game). 

First, we get the data. Make sure that the file 'Pokemon151.jpg' is in the same folder as this Jupyter notebook.

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Load the image
image_path = './Pokemon151.jpg'
image = np.array(Image.open(image_path).convert('L'))
data = []

# Cut the image to get pictures of individual Pokémon
for i in range(10):
    if i != 9:
        jmax = 16
    else:
        jmax = 7
    for j in range(jmax):
        data.append(image[i*46:(i+1)*46,j*46:(j+1)*46]/255.)
# We create a flattened version of the data (i.e., every image becomes a vector)
# for later use in the algorithm.
flattened_data = np.array(data).reshape(151, -1)

# Look at the pictures (note: there is noise!)
plt.imshow(data[150], cmap='grey')
_ = plt.axis('off')

**Here, you have to implement the PCA method.**

As a reminder, we require four steps:

1) For each pixel, we calculate the mean over all data samples (here: 151 images of Pokémon). For convenience, we store these values in a vector of lengthh $46^2$.
2) Subtract the mean from each data sample (element-wise). This way, we have $x_n - \mu$, where $x_n$ is a vector storing the $n^\text{th}$ image and $\mu$ the mean vector.
3) With this, we calculate the covariance matrix (averaged over all data samples). To get the covariance matrix, we have to form the outer product $(x_n - \mu)(x_n - \mu)^T$.
4) Get the eigenvectors of the covariance matrix that correspond to the two largest eigenvalues. This is already implemented here using 'np.linalg.eigh'.

In [ ]:
## YOUR CODE GOES HERE!

# Calculate the mean value of each pixel (choose 'axis' correctly in np.mean!)
# Use 'flattened_data' for this!
mean_vector = 
# Subtract the mean from 'flattened_data'
# Use 'centered_data' here, as we will use this vector later as well
centered_data = 

# Calculate the covariance matrix (np.outer is useful for this!)
covariance = 

# Spectral decomposition of the covariance matrix
eigenvalues, eigenvectors = np.linalg.eigh(covariance)

# Sort the eigenvalues
idx = np.argsort(eigenvalues)[::-1]
# Get the two eigenvectors with the largest eigenvalues 
largest_eigvecs = eigenvectors[:, idx[:2]]

First, we can have a look at the eigenvalues of the covariance matrix. They are sorted in descending order, with the first eigenvalue being, by far, the largest.

In [ ]:
import seaborn as sns

plt.plot(eigenvalues[::-1], linewidth=0, color='steelblue', marker='o', markersize=7.5)
plt.xlim(-1,30)
plt.ylabel('Value', fontsize = 20)
plt.xlabel('Eigenvalue index', fontsize = 20)
sns.despine()
plt.tick_params(axis='both', which='major', labelsize=18)

We can also visualise the eigenvectors by reshaping them back into image shapes! Pokémon have quite diverse shapes, so it is hard to identify "high-level features" in the images. However, you can spot some shades resembling Pokémon in the images.

In [ ]:
fig, ax = plt.subplots(4,4, figsize=(16,16))
for j in range(4):
    for i in range(4):
        ax[j][i].imshow(np.reshape(eigenvectors[:, idx[j*4+i]], (46, 46)), cmap = 'gray')
        ax[j][i].set_yticks([])
        ax[j][i].set_xticks([])
        ax[j][i].set_xlabel('Eigenvalue index: ' + str(idx[j*4+i]), fontsize = 16)

Finally, we use the two eigenvectors with largest eigenvalue to project each data sample into a two-dimensional space! This way, we can visualise the $46 \times 46$ dimensional images in only two dimensions.

In [ ]:
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import seaborn as sns

projected_data = []
for i in range(len(data)):
    projected_data.append(centered_data[i] @ largest_eigvecs)
projected_data = np.array(projected_data)

def imscatter(x, y, images, ax=None, zoom=0.1):
    """
    x, y     : coordinates
    images   : list/array of image arrays (numpy or PIL)
    ax       : matplotlib axis
    zoom     : size of thumbnails
    """
    if ax is None:
        _, ax = plt.subplots(figsize=(20, 10))

    artists = []
    for (x0, y0, img) in zip(x, y, images):
        imagebox = OffsetImage(img, zoom=zoom, cmap='grey')
        ab = AnnotationBbox(imagebox, (x0, y0), frameon=False)
        artists.append(ax.add_artist(ab))
    return artists

_ = imscatter(projected_data[:,0], projected_data[:,1], data, zoom=.9)
plt.xlim((-10,12))
plt.ylim((-12,10.5))
plt.xlabel('First principal component', fontsize = 20)
plt.ylabel('Second principal component', fontsize = 20, labelpad=20)
plt.tick_params(axis='both', which='major', labelsize=18)
sns.despine()

### 2. Marcenko-Pastur distribution

We will have a brief look at the Marcenko-Pastur distribution. As in the lecture, we generate data points from a Gaussian distribution, and we "spike" the correlation matrix into some direction. If this perturbation is strong enough, it will manifest as an eigenvalue outside of the Marcenko-Pastur distribution.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
np.random.seed(42)

def marcenko_pastur(x, gamma):
    # YOUR CODE GOES HERE!
    # Add the formula of the Marcenko-Pastur distribution
    gammap = 
    gammam = 

    return 

# Strength of the component we add
beta = 1.5
# Variance of random data
sigma = 1
# Dimension of data points
dim = 2000
# Number of data points
num_samples = 3000
# Gamma as defined in the lecture
gamma = dim/num_samples
    
# The principal component we are adding
component = np.ones(dim)
component = component/np.linalg.norm(component)

# Generation of random samples with principal component
mean = np.zeros(dim)
cov = np.eye(dim) + beta *np.outer(component, component)
points = np.random.multivariate_normal(mean, cov, num_samples)

# YOUR CODE GOES HERE!
# Calculate the covariance matrix from the data points and
# get its eigenvalues and eigenvectors (just like for PCA)
covariance = 
eigenvalues, eigenvectors = np.linalg.eigh(covariance)

plt.figure()
_ = plt.hist(eigenvalues, bins = 200, density=True)

vals = np.linspace(0,12,10000)
plt.plot(vals, marcenko_pastur(vals, gamma), linewidth=2.5, color = 'tomato')

plt.ylabel(r'$p(\lambda)$', fontsize=20)
plt.xlabel(r'$\lambda$', fontsize=20)

plt.tick_params(axis='both', which='major', labelsize=18)

plt.xlim(-0.005,5)
sns.despine()

### 3. Random projections

Time to look at the power of random projections! As we saw in the lecture, random projections can have strong guarantees for conserving vector lengths and vector distances. This is especially useful if we are faced with very high-dimensional data, and we want to find or group data points that are similar to each other. We can perform these operations in the projected subspace!

We will evaluate the Johnson-Lindenstrauss lemma here numerically. First, we load the famous MNIST handwritten digits dataset using the pytorch library!

In [ ]:
import torch 
import numpy as np
from torchvision import datasets, transforms

# load mnist data from pytorch
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())

# data  loader
trainloader = torch.utils.data.DataLoader(mnist_trainset, batch_size=200, shuffle=True)

Lets visualise some data samples.

In [ ]:
fig, ax = plt.subplots(1,20, figsize=(20,5))

for i in range(20):
    ax[i].imshow(mnist_trainset[i][0][0], cmap='gray_r')
    ax[i].axis('off')

Time to project the data! You can change the parameters to adjust the quality of the projections!

In [ ]:
# Number of samples
m = 200
# Error
epsilon = 0.5
# Dimension of the subspace
d = int(4*np.log(m)/epsilon**2)
# Probability that distances and lengths are preserved
prob = 1-2/m 
print('At dimension {}, lengths and distances of data points are preserved with probability at least {}.'.format(d, prob))

# Create random projections
np.random.seed(4242)
# YOUR CODE GOES HERE!
# Create the projection matrix as in the lecture (random numbers from a Gaussian with zero mean and variane 1, rescaled by 1/sqrt(d))
projection_operator = 

# data loader, we will only use the first batch of size "m"
trainloader = torch.utils.data.DataLoader(mnist_trainset, batch_size=m, shuffle=True)

# For every data point in the dataset (trainloader), project it using M.
for data, _ in trainloader:
    # Some pytorch magic... we switch to numpy here for simplicity. Torch is only used to get the data.
    data = data.squeeze(1)
    data = data.view(-1, 28*28)
    data = data.numpy()
    # YOUR CODE GOES HERE!
    # Project the data using the projection_operator
    transformed = 
    break

To evaluate the results, we need the differences between the original data points, as well as the differences between the projected data points.

In [ ]:
diffs_theory = []
diffs_actual = []
for i in range(m):
    for j in range(i+1, m):
        # YOUR CODE GOES HERE!
        # Calculate the distance between the projected data points
        diffs_theory.append(...)
        # Calculate the distance between the original data points
        diffs_actual.append(...)

Lets plot the results! The dashed lines are the bounds of the Johnson-Lindenstrauss lemma.

In [ ]:
import matplotlib.pyplot as plt

# We look at the ratio of the length of the original data points and their projected versions
plt.plot((np.linalg.norm(transformed, axis=1)/np.linalg.norm(data, axis=1)), color='steelblue', linewidth = 2.5)
plt.hlines(1+epsilon, 0, m-1, color = 'k', linewidth=2.5, linestyles='--')
plt.hlines(1-epsilon, 0, m-1, color = 'k', linewidth=2.5, linestyles='--')
plt.tick_params(axis='both', which='major', labelsize=18)
plt.xlabel('Training samples', fontsize=20)
plt.show()

# Same for the distances between data points
plt.plot(np.array(diffs_theory)/np.array(diffs_actual), color='steelblue', linewidth = 2.5)
plt.hlines(1+epsilon, 0, len(diffs_actual), color = 'k', linewidth=2.5, linestyles='--')
plt.hlines(1-epsilon, 0, len(diffs_actual), color = 'k', linewidth=2.5, linestyles='--')
plt.tick_params(axis='both', which='major', labelsize=18)
plt.xlabel('Pairwise combinations', fontsize=20)
plt.show()

### 4. k-means clustering

Time to do some clustering. First, we will implement one of the most classic clustering algorithms ourselves. For convex clusters, this algorithm works well. However, as we will see, for non-convex ones, it struggles. To improve the method, we will upgrade it in the section that follows using spectral clustering! **In this part, your task is to complete the k-means function!**

First, we get some data to play with. We get these from the Python library sklearn.

In [ ]:
from sklearn import datasets
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
import numpy as np
import seaborn as sns

n_samples = 500
seed = 30
noisy_circles = datasets.make_circles(
    n_samples=n_samples, factor=0.5, noise=0.05, random_state=seed
)

varied = datasets.make_blobs(
    n_samples=n_samples, cluster_std=[1.0, 2.5, 0.5], random_state=442
)

Lets have a look at this data! For simplicity, it is only two-dimensional data. By eye, we can immediately identify "clusters" of data that belong together.

In [ ]:
plt.figure()
plt.scatter(noisy_circles[0][:,0], noisy_circles[0][:,1], edgecolors='k', s=200)
plt.title('Noisy circles', fontsize= 14)
sns.despine()

plt.figure()
plt.scatter(varied[0][:,0], varied[0][:,1], edgecolors='k', s=200)
plt.title('Varied convex clusters', fontsize= 14)
sns.despine()

In [ ]:
def kmeans(dataset, num_clusters, iters = 10, dim=2, seed = 4242):
    np.random.seed(seed)

    ## INITIALISATION

    # clusters_mean contains the mean vector of each cluster
    # we set it to 0 for now and then initialise it
    clusters_mean = np.zeros((num_clusters, dim))
    # YOUR CODE GOES HERE
    # Assign random clusters to all points
    # For instance, if we try to find 3 clusters, we assign each
    # data point a label in [0,1,2]
    random_init = ...
    # Calculate the mean vector for each cluster
    # I.e., cluster_mean[0] = mean of all data points assigned label 0 etc. 
    ...

    # K-MEANS

    # Lets start with the algorithm! We loop 'iter' amount of times (to avoid endless loops...)
    for _ in range(iters):
        # labels contains the cluster assignment
        labels = []

        # loop over all data points
        for data_sample in dataset:
            # YOUR CODE GOES HERE!
            # For data_sample, calculate its distance to all cluster centres (clusters_mean)
            distances = ...
            # Assign the label corresponding to the cluster centre that is closest
            # I.e., we are looking for the argmin of distances!
            labels.append(...)
        labels = np.array(labels)
        
        # Recalculate the mean vector for each cluster
        # HERE GOES the same code you wrote for the initialisation, but
        # we use 'labels' now!
        for i in range(num_clusters):
            if len(dataset[np.array(labels)==i]) != 0:
                clusters_mean[i] = np.mean(dataset[np.array(labels)==i], axis=0)

    return labels

Lets try it!

In [ ]:
dataset = varied[0]
num_clusters = 3

clusters = kmeans(dataset, num_clusters)

colors = ['orange', 'tomato', 'steelblue']
for i in range(num_clusters):
    data_cluster = dataset[np.array(clusters)==i]
    plt.scatter(data_cluster[:,0], data_cluster[:,1], color=colors[i], edgecolors='k', s=200)
    plt.axis('off')

You should see three nicely identified clusters! :) Lets try the other dataset!

In [ ]:
dataset = noisy_circles[0]
num_clusters = 2

clusters = kmeans(dataset, num_clusters)

colors = ['orange', 'tomato', 'steelblue']
for i in range(num_clusters):
    data_cluster = dataset[np.array(clusters)==i]
    plt.scatter(data_cluster[:,0], data_cluster[:,1], color=colors[i], edgecolors='k', s=200)
    plt.axis('off')

That... did not work :( But we can fix this with a simple adjustment!

### 5. Spectral clustering

In spectral clustering, we first find a new representation of our data coming from the spectral decomposition of the graph Laplacian. **Your task is to upgrade the k-means method you implemented to spectral clustering!**

In [ ]:
# First, calculate the similarity between data points
# Gamma is a hyperparameter that controls how far points can be apart from each other and still be considered similar
gamma = 0.1 # For varied, gamma = 1 works well. For circles, gamma = 0.1
dataset = noisy_circles[0]
num_clusters = 2

# YOUR CODE GOES HERE
# Calculate the similarity of data points
# We use a Gaussian kernel for this: similarity(x,y) = exp(-(x-y))^2/(2*gamma^2)
distances = ...
# Make sure that the diagonal of distances is 0!
...
# Now lets calculate the degree matrix
degree = ...
# The Laplacian is then obtained by taking the difference
laplacian = degree - distances

# All that remains to be done is the spectral decomposition. 
eigenvalues, eigenvectors = np.linalg.eigh(laplacian)
sorted_ids = np.argsort(eigenvalues)
# Different from PCA, we take the eigenvectors with the smallest eigenvalues!
transformed_data = eigenvectors[:, :num_clusters]

# And now, we simply do k-means using our transformed data!
clusters = kmeans(transformed_data, num_clusters, dim=num_clusters)

colors = ['orange', 'tomato', 'steelblue']
for i in range(num_clusters):
    data_cluster = dataset[np.array(clusters)==i]
    plt.scatter(data_cluster[:,0], data_cluster[:,1], color=colors[i], edgecolors='k', s=200)
    plt.axis('off')

### 6. Diffusion maps / Laplacian Eigenmaps

As a last exercise, let's implement diffusion maps! **Your task is to fill out the missing code for constructing the Laplacian!**

First, load some interesting data. We will use the S-curve in the following

In [ ]:
from sklearn.datasets import make_s_curve
import matplotlib.pyplot as plt

dataset, colors = make_s_curve(n_samples=2000, random_state = 42)

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection="3d")
fig.add_axes(ax)
ax.scatter(
    dataset[:, 0], dataset[:, 1], dataset[:, 2], c=colors, s=50, alpha=0.8
)
ax.view_init(azim=-66, elev=12)

Second, we implement the algorithm and plot our results :)

In [ ]:
import numpy as np
from scipy.spatial.distance import cdist
# First, calculate the similarity between data points
# Gamma is a hyperparameter that controls how far points can be apart from each other and still be considered similar
gamma = .2 # For varied, gamma = 1 works well. For circles, gamma = 0.1
reduced_dim = 2

# YOUR CODE GOES HERE
# Calculate the similarity of data points
# We use a Gaussian kernel for this: similarity(x,y) = exp(-(x-y))^2/(2*gamma^2)
# (This is the same as for Spectral Clustering)
distances = 
# Make sure that the diagonal of distances is 0!

# Now lets calculate the degree matrix
degree = 
# And its inverse
degree_inv = 

# The Laplacian is obtained by the matrix product of degree_inv and distances
M = 
# The symmetrised Laplacian is given by
S = np.sqrt(degree) @ M @ np.sqrt(degree_inv)

# All that remains to be done is the spectral decomposition. 
eigenvalues, eigenvectors = np.linalg.eigh(S)
eigenvectors = np.sqrt(degree_inv) @ eigenvectors
# # Different from PCA, we take the eigenvectors with the smallest eigenvalues!
transformed_data = eigenvectors[:, -3:-1]
transformed_data[:, 0] = eigenvalues[-3] * transformed_data[:,0]
transformed_data[:, 1] = eigenvalues[-2]* transformed_data[:,1]

# Time to plot our result!
_ = plt.scatter(transformed_data[:,0], transformed_data[:,1], c=colors, s=50, alpha=0.8)
_ = plt.axis('off')